Kakao Developers의 REST API 키, Authorization, 쿼리 파라미를 사용한 데이터 수집

In [42]:
# 필요한 모듈 설치
import requests
import json
from pandas import DataFrame
# 데이터 조작, 필터링, 정렬, 그룹화, 병합 등 수행 가능

In [43]:
# 접속할 URL 설정(urlFmt는 urlformat의 약자)
urlFmt = "https://dapi.kakao.com/v3/search/book?query={query}&page={page}&size={size}"

In [44]:
# 요청변수(URL 변수)
# #검색을 원하는 질의어
query = "은혼"
# # 결과 페이지 번호
page = 1
# # 페이지 당 보여질 문서 수
size = 50
# # kakao developers에서 발급 받은 REST API 키
key = "3b23dfd1bf5a535f83c2792589d13d21"

In [45]:
# 접속 객체 생성
session = requests.Session()

# 접속 객체에 부가정보(header) 삽입
# daum 검색에서는 "Referer"과 "User-Agent"를 설정해줄 필요가 없음
# key는 위에서 발급 받은 REST API 키
session.headers.update({
    "Authorization" : "KakaoAK %s" %key
})

In [46]:
# 다음 페이지가 존재하는지 여부 --> False인 동안은 while문 반복
isEnd = False

# 반복 수행 도중 추출되는 검색결과 리스트를 결합할 빈 리스트 준비
mylist = []

# 1페이지부터 시작
page = 1

while (not isEnd): # while은 'True일 때' 반복이 진행되기 때문에 not을 붙여준다 
    #--------------------------------------------------------------
    # 이전 예제 코드 시작
    #--------------------------------------------------------------
    # 접속할 URL
    url = urlFmt.format(query=query, page=page, size=size)
    url

    # 생성한 접속객체를 활용하여 API에 접속
    r = session.get(url)

    # 접속에 실패한 경우
    if r.status_code != 200:
        # 에러코드와 에러메시지 출력
        msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
        # 에러를 강제로 생성시킴
        raise Exception(msg)

    # 인코딩 형식 지정
    r.encoding = "utf-8"
    # 가져온 문자열을 딕셔너리로 변환
    data = json.loads(r.text)
    #data
    #--------------------------------------------------------------
    # 이전 예제 코드 끝
    #--------------------------------------------------------------

    # 검색 결과를 갖고 있는 딕셔너리를 포함하는 리스트 추출
    searchList = data['documents']

    # 추출한 리스트를 미리 준비한 빈 리스트에 추가
    mylist.extend(searchList)

    # 증감식에 해당하는 isEnd변수 추출
    isEnd = bool(data['meta']['is_end'])
    page += 1
    print("isEnd=", isEnd, ", 다음페이지 =", page)

print("최종 검색 건수: %d" % len(mylist))

isEnd= False , 다음페이지 = 2
isEnd= False , 다음페이지 = 3
isEnd= False , 다음페이지 = 4
isEnd= False , 다음페이지 = 5
isEnd= False , 다음페이지 = 6
isEnd= False , 다음페이지 = 7
isEnd= False , 다음페이지 = 8
isEnd= True , 다음페이지 = 9
최종 검색 건수: 391


In [47]:
# 책 가격 결과를 엑셀 형태로 저장 및 출력

# 책 가격을 담을 초기 list
sale_prices = []
# mylist에서 각 항목의 sale_price를 추출하여 리스트에 추가
for item in mylist:
    sale_prices.append(item['sale_price'])

# 추출한 sale_price를 엑셀 파일로 저장
df = DataFrame(sale_prices)
df.to_excel("은혼의 모든 책 가격.xlsx", index=False)
df

,0
0,7200
1,7200
2,4050
3,4050
4,4050
...,...
386,-1
387,-1
388,-1
389,-1


In [48]:
# '은혼'이라는 키워드로 검색된 모든 책의 총 가격은 얼마인가?
# query = "은혼"
# for문을 사용해 price만 추출
# for item in mylist:
#     print(item['sale_price'])